In [2]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy

from steps.prepare_data import load_processed_data
from utils.model import predict, load_model

In [3]:
# Balance UA - перетворити в boolean взяти який поріг

train_data, test_data = load_processed_data()

train_data_x = train_data.drop(columns=["target"])

In [4]:
train_data_x_with_mnt_wk = train_data_x.filter(regex="mnt|wk")

columns = train_data_x_with_mnt_wk.columns

df = pd.DataFrame(columns, columns=["column_name"])

df["index"] = df["column_name"].apply(lambda x: "_".join(x.split("_")[0:-1]))

df = df.set_index("index")

df.sort_index(inplace=True)

# pd.set_option("display.max_rows", None)

df.sort_values(by="column_name")

,column_name
index,
Ama_rchrgmnt_sum_max,Ama_rchrgmnt_sum_max_mnt1
Ama_rchrgmnt_sum_max,Ama_rchrgmnt_sum_max_mnt3
Ama_rchrgmnt_sum_mea,Ama_rchrgmnt_sum_mea_mnt1
Ama_rchrgmnt_sum_mea,Ama_rchrgmnt_sum_mea_mnt3
Ama_rchrgmnt_sum_mea,Ama_rchrgmnt_sum_mea_wk1
...,...
voice_out_td_cnt_min,voice_out_td_cnt_min_mnt1
voice_out_td_cnt_min,voice_out_td_cnt_min_mnt3
voice_out_td_cnt_std,voice_out_td_cnt_std_mnt1


In [46]:
df_agg_columns = df.groupby("index").count().sort_values(by="column_name", ascending=False)

slope_columns = df_agg_columns[df_agg_columns.column_name == 3]
slope_columns[slope_columns.index.str.endswith("_mea")].index

Index(['pay_sum_mea', 'conn_in_uniq_cnt_mea', 'gprs_tar_vol_mea',
       'conn_com_part_mea', 'voice_mts_in_nrest_part_mea',
       'voice_in_cmpttrs_td_cnt_mea', 'com_num_part_mea', 'ks_num_part_mea',
       'voice_mts_in_nwork_part_mea', 'clc_no_vas_roam_mea',
       'non_accum_internet_vol_mea', 'voice_in_mts_avg_dur_mea', 'vas_clc_mea',
       'ama_volume_mea', 'voice_mts_in_dwork_part_mea',
       'voice_in_fix_tar_dur_mea', 'num_act_days_mea',
       'voice_mts_in_drest_part_mea', 'data_3g_tar_vol_mea',
       'data_3g_tv_cnt_mea', 'voice_in_roam_clc_mea', 'data_3g_dou_mea',
       'voice_in_life_part_mea', 'voice_in_short_part_mea', 'goodok_clc_mea',
       'content_cnt_mea', 'voice_in_tar_dur_mea', 'voice_in_kievstar_part_mea',
       'content_clc_mea', 'Ama_rchrgmnt_sum_mea', 'voice_in_td_cnt_mea',
       'gprs_clc_mea', 'conn_out_uniq_cnt_mea', 'block_all_dur_mea',
       'all_cnt_mea', 'sms_clc_mea', 'voice_out_short_part_mea',
       'sms_in_cnt_mea', 'pay_p2p_in_sum_mea', 

In [79]:
# pd.reset_option("display.max_rows")

# train_data_x_with_mnt_wk['num_act_days_mea_mnt3'].idxmax()

# train_data[train_data.target == 1].iloc[12][["num_act_days_mea_mnt3", "num_act_days_mea_mnt1", "num_act_days_mea_wk1"]]

# train_data[(train_data.target == 1)].corr()
# lost_clients = train_data[(train_data.target == 1)]

In [52]:
def calculate_slopes(df, column):
    t = np.array([1, 2, 3])
    columns = [f"{column}_mnt3", f"{column}_mnt1", f"{column}_wk1"]
    y = df[columns].fillna(method="ffill", axis=1).values.T
    slopes = np.apply_along_axis(lambda x: scipy.stats.linregress(t, x)[0], 0, y)
    return slopes


for column in slope_columns.index:
    train_data[column] = calculate_slopes(train_data, column)
    train_data.drop(columns=[f"{column}_mnt3"], inplace=True)

/var/folders/rx/3wq3smm97tngqbtszstwnf4c0000gn/T/ipykernel_52208/1929101703.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = df[columns].fillna(method="ffill", axis=1).values.T
/var/folders/rx/3wq3smm97tngqbtszstwnf4c0000gn/T/ipykernel_52208/1929101703.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[column] = calculate_slopes(train_data, column)
/var/folders/rx/3wq3smm97tngqbtszstwnf4c0000gn/T/ipykernel_52208/1929101703.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = df[columns].fillna(method="ffill", axis=1).values.T
/var/folders/rx/3wq3smm97tng

KeyboardInterrupt: 

24        1
30        0
60        1
77        0
104       0
         ..
149964    0
149971    0
149976    0
149981    0
149992    0
Name: loc_is_obl_center, Length: 9585, dtype: int64